In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
from simility_apis.return_rule_info_from_simility import ReturnRuleInfoFromSimilityAPI
from simility_requests.return_rule_info_from_simility import ReturnRuleInfoFromSimility
# from simility_requests.return_cassandra_dtypes import ReturnCassandraDatatypes
from simility_apis.set_password import set_password
import pandas as pd
import numpy as np
import json

In [18]:
set_password()
# 083025B41
params = {
    'url': 'http://sim-ds.us-central1.gcp.dev.paypalinc.com',
    'app_prefix': 'james_testing',
    'user': 'james@simility.com',
    'entity': 'transaction'
}
rri = ReturnRuleInfoFromSimilityAPI(**params)
# rri = ReturnRuleInfoFromSimility(**params)
rule_info = rri.request()

Please provide your password for logging into the Simility platform:  ·········


In [19]:
rule_configs_list = json.loads(rule_info)['entries']

In [20]:
rule_configs_dict = dict((rule_config['name'], rule_config) for rule_config in rule_configs_list)

In [21]:
rule_configs_dict.keys()

dict_keys(['HighOrderAmount', 'HighTransactionPerAccount', 'BillingShippingCityMismatchAndHighOrderItems', 'CreateRuleTest1', 'CreateRuleTest2', 'Rule1', 'Rule2', 'Rule3'])

In [22]:
rule_configs_dict

{'HighOrderAmount': {'id': 240,
  'modifiedBy': 'james@simility.com',
  'modifiedOn': '2020-12-16T04:12:05.729-08:00',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'HighOrderAmount',
  'casesCaughtByRule': 0,
  'conditions': '{"condition": "AND", "rules": [{"field": "order_total", "operator": "greater_or_equal", "value": "value.100.0", "id": "order_total", "type": "double"}]}',
  'conditionsCleaned': '{"condition":"AND","rules":[{"id":"order_total","field":"order_total","type":"double","operator":"greater_or_equal","value":"value.100.0"}]}',
  'conditionsJson': {'condition': 'AND',
   'rules': [{'field': 'order_total',
     'id': 'order_total',
     'operator': 'greater_or_equal',
     'type': 'double',
     'value': 'value.100.0'}]},
  'conditionsV2': '{"operator":"AND","rules":[{"operator":"GREATER_THAN_OR_EQUAL","rules":[{"type":"DOUBLE","operator":"TABLE_VALUE","value":"order_total"},{"type":"DOUBLE","operator":"CONSTANT","value":"100.0"}]}]}',
  'condit

In [23]:
original_configs = rule_configs_dict.copy()

In [24]:
new_rule_strings = {
    'HighOrderAmount': "(X['order_total']>=50)",
    'HighTransactionPerAccount': "(X['account_number_num_distinct_transaction_per_account_number_1day']>=2)&(X['account_number_num_distinct_transaction_per_account_number_7day']>=4)&(X['account_number_num_distinct_transaction_per_account_number_30day']>=6)",
    'BillingShippingCityMismatchAndHighOrderItems': "(X['is_billing_shipping_city_same']==False)&(X['num_order_items']>=2)",
    'CreateRuleTest1': "(X['order_total']>=100)",
    'CreateRuleTest2': "(X['order_total']>=250)",
    'Rule1': "(X['account_number_avg_order_total_per_account_number_1day']>=2)",
    'Rule2': "(X['sim_sc_ml']>=0.75)",
    'Rule3': "(X['is_billing_shipping_city_same']==True)"
}

In [25]:
field_datatypes = {
    'account_number_avg_order_total_per_account_number_1day': 'DOUBLE',
    'sim_sc_ml': 'FLOAT',
    'is_billing_shipping_city_same': 'BOOLEAN',
    'order_total': 'DOUBLE',
    'num_order_items': 'INT',
    'account_number_num_distinct_transaction_per_account_number_1day': 'INT',
    'account_number_num_distinct_transaction_per_account_number_7day': 'INT',
    'account_number_num_distinct_transaction_per_account_number_30day': 'INT',
    
}
cassandra_field_names = {
    'account_number_avg_order_total_per_account_number_1day': 'account_number.avg_order_total_per_account_number_1day',
    'sim_sc_ml': 'sim_sc_ml',
    'is_billing_shipping_city_same': 'is_billing_shipping_city_same',
    'order_total':'order_total',
    'num_order_items':'num_order_items',
    'account_number_num_distinct_transaction_per_account_number_1day':'account_number.num_distinct_transaction_per_account_number_1day',
    'account_number_num_distinct_transaction_per_account_number_7day':'account_number.num_distinct_transaction_per_account_number_7day',
    'account_number_num_distinct_transaction_per_account_number_30day':'account_number.num_distinct_transaction_per_account_number_30day',
}

In [26]:
from rules.rules import Rules

In [27]:
r = Rules(rule_strings=new_rule_strings)

In [28]:
new_system_conditions = r.as_system_dicts(field_datatypes=field_datatypes, cassandra_field_names=cassandra_field_names)

In [29]:
new_system_conditions

{'HighOrderAmount': {'condition': 'AND',
  'rules': [{'field': 'order_total',
    'operator': 'greater_or_equal',
    'value': 'value.50.0',
    'id': 'order_total',
    'type': 'double'}]},
 'HighTransactionPerAccount': {'condition': 'AND',
  'rules': [{'field': 'account_number.num_distinct_transaction_per_account_number_1day',
    'operator': 'greater_or_equal',
    'value': 'value.2',
    'id': 'account_number.num_distinct_transaction_per_account_number_1day',
    'type': 'integer'},
   {'field': 'account_number.num_distinct_transaction_per_account_number_7day',
    'operator': 'greater_or_equal',
    'value': 'value.4',
    'id': 'account_number.num_distinct_transaction_per_account_number_7day',
    'type': 'integer'},
   {'field': 'account_number.num_distinct_transaction_per_account_number_30day',
    'operator': 'greater_or_equal',
    'value': 'value.6',
    'id': 'account_number.num_distinct_transaction_per_account_number_30day',
    'type': 'integer'}]},
 'BillingShippingCityM

In [31]:
new_scores = {
    'HighOrderAmount': -50,
    'HighTransactionPerAccount': -50,
    'BillingShippingCityMismatchAndHighOrderItems': -50,
    'CreateRuleTest1': -50,
    'CreateRuleTest2': -50,
    'Rule1': -50,
    'Rule2': -50,
    'Rule3': -50,
}

In [32]:
from system_config_generation.update_existing_configs import UpdateExistingConfigs

In [33]:
up = UpdateExistingConfigs(rule_configs=rule_configs_dict, updated_conditions=new_system_conditions, updated_scores=new_scores)

In [34]:
updated_rule_configs = up.update()

In [35]:
rule_configs_dict['HighOrderAmount']

{'id': 240,
 'modifiedBy': 'james@simility.com',
 'modifiedOn': '2020-12-16T04:12:05.729-08:00',
 'appPrefix': 'james_testing',
 'entityName': 'transaction',
 'name': 'HighOrderAmount',
 'casesCaughtByRule': 0,
 'conditions': '{"condition": "AND", "rules": [{"field": "order_total", "operator": "greater_or_equal", "value": "value.100.0", "id": "order_total", "type": "double"}]}',
 'conditionsCleaned': '{"condition":"AND","rules":[{"id":"order_total","field":"order_total","type":"double","operator":"greater_or_equal","value":"value.100.0"}]}',
 'conditionsJson': {'condition': 'AND',
  'rules': [{'field': 'order_total',
    'id': 'order_total',
    'operator': 'greater_or_equal',
    'type': 'double',
    'value': 'value.100.0'}]},
 'conditionsV2': '{"operator":"AND","rules":[{"operator":"GREATER_THAN_OR_EQUAL","rules":[{"type":"DOUBLE","operator":"TABLE_VALUE","value":"order_total"},{"type":"DOUBLE","operator":"CONSTANT","value":"100.0"}]}]}',
 'conditionsV2Json': {'operator': 'AND',
  '

In [36]:
updated_rule_configs

{'HighOrderAmount': {'id': 240,
  'modifiedBy': 'argo@simility.com',
  'modifiedOn': '2020-12-21T15:42:11',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'HighOrderAmount',
  'casesCaughtByRule': 0,
  'conditions': '{"condition": "AND", "rules": [{"field": "order_total", "operator": "greater_or_equal", "value": "value.50.0", "id": "order_total", "type": "double"}]}',
  'entitiesProcessed': 0,
  'isAutoGenerated': True,
  'javaElExpression': '',
  'lastDateRuleConditionChanged': '2020-12-16T04:12:05.732-08:00',
  'matches': 0,
  'query': '',
  'rid': 240,
  'ruleGroupNames': 'Test',
  'score': -50,
  'status': 'ACTIVE',
  'totalCases': 0,
  'totalEntities': 0},
 'HighTransactionPerAccount': {'createdBy': 'james@simility.com',
  'createdOn': '2020-03-10T13:19:10.938-07:00',
  'id': 383,
  'modifiedBy': 'argo@simility.com',
  'modifiedOn': '2020-12-21T15:42:11',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'HighTransactionPerAccount',

In [37]:
from simility_requests.update_rules_in_simility import UpdateRulesInSimility

In [38]:
up_rules = UpdateRulesInSimility(url=params['url'], app_prefix=params['app_prefix'], user=params['user'])

In [39]:
up_rules.request(rule_configs=updated_rule_configs)

# Unit testing

In [10]:
rule_configs = {'Rule1': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule1',
                              'conditions': '{"condition": "AND", "rules": [{"condition": "OR", "rules": [{"id": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "type": "double", "operator": "greater_or_equal", "value": "value.60.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "type": "double", "operator": "greater", "value": "value.120.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "type": "double", "operator": "less_or_equal", "value": "value.500.0"}]}, {"id": "num_items", "field": "num_items", "type": "integer", "operator": "equal", "value": "value.1"}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True},
                    'Rule2': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule2',
                              'conditions': '{"condition": "AND", "rules": [{"id": "ml_cc_v0", "field": "ml_cc_v0", "type": "double", "operator": "less", "value": "value.0.315"}, {"condition": "OR", "rules": [{"id": "method_clean", "field": "method_clean", "type": "string", "operator": "equal", "value": "value.checkout"}, {"id": "method_clean", "field": "method_clean", "type": "string", "operator": "begins_with", "value": "value.checkout"}, {"id": "method_clean", "field": "method_clean", "type": "string", "operator": "ends_with", "value": "value.checkout"}, {"id": "method_clean", "field": "method_clean", "type": "string", "operator": "contains", "value": "value.checkout"}, {"id": "ip_address", "field": "ip_address", "type": "string", "operator": "is_not_null", "value": null}, {"id": "ip_isp", "field": "ip_isp", "type": "string", "operator": "is_not_empty", "value": null}]}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True},
                    'Rule3': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule3',
                              'conditions': '{"condition": "AND", "rules": [{"id": "method_clean", "field": "method_clean", "type": "string", "operator": "not_begins_with", "value": "value.checkout"}, {"id": "method_clean", "field": "method_clean", "type": "string", "operator": "not_ends_with", "value": "value.checkout"}, {"id": "method_clean", "field": "method_clean", "type": "string", "operator": "not_contains", "value": "value.checkout"}, {"condition": "OR", "rules": [{"id": "ip_address", "field": "ip_address", "type": "string", "operator": "is_null", "value": null}, {"id": "ip_isp", "field": "ip_isp", "type": "string", "operator": "is_empty", "value": null}]}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True},
                    'Rule4': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule4',
                              'conditions': '{"condition": "AND", "rules": [{"id": "forwarder_address", "field": "forwarder_address", "type": "boolean", "input": "radio", "operator": "equal", "value": "1"}, {"id": "is_shipping_billing_address_same", "field": "is_shipping_billing_address_same", "type": "boolean", "input": "radio", "operator": "equal", "value": "0"}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True},
                    'Rule5': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule5',
                              'conditions': '{"condition": "AND", "rules": [{"id": "ad_price_type", "field": "ad_price_type", "type": "string", "operator": "not_in", "value": "value.FREE,NEGOTIATION"}, {"id": "ad_price_type", "field": "ad_price_type", "type": "string", "operator": "in", "value": "value.FOO,BAR"}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True},
                    'Rule6': {'createdBy': 'argo@simility.com',
                              'createdOn': '2020-12-16T15:00:17',
                              'appPrefix': 'james_testing',
                              'entityName': 'transaction',
                              'name': 'Rule6',
                              'conditions': '{"condition": "AND", "rules": [{"id": "ip_country_iso_code", "field": "ip_country_iso_code", "type": "string", "operator": "equal", "value": "field.billing_country"}, {"id": "country_id", "field": "country_id", "type": "string", "operator": "not_equal", "value": "field.ip_country_iso_code"}]}',
                              'score': -100,
                              'status': 'ACTIVE',
                              'isAutoGenerated': True}}

In [11]:
new_conditions = {'Rule1': {'condition': 'AND',
                                'rules': [{'condition': 'OR',
                                           'rules': [{'id': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                      'field': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                      'type': 'double',
                                                      'operator': 'greater_or_equal',
                                                      'value': 'value.120.0'},
                                                     {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                      'field': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                      'type': 'double',
                                                      'operator': 'greater',
                                                      'value': 'value.240.0'},
                                                     {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                      'field': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                      'type': 'double',
                                                      'operator': 'less_or_equal',
                                                      'value': 'value.1000.0'}],
                                           },
                                          {'id': 'num_items',
                                           'field': 'num_items',
                                           'type': 'integer',
                                           'operator': 'equal',
                                           'value': 'value.2'},
                                          ]},
                      'Rule2': {'condition': 'AND',
                                'rules': [{'id': 'ml_cc_v0',
                                           'field': 'ml_cc_v0',
                                           'type': 'double',
                                           'operator': 'less',
                                           'value': 'value.0.75'},
                                          {'condition': 'OR',
                                           'rules': [{'id': 'method_clean',
                                                      'field': 'method_clean',
                                                      'type': 'string',
                                                      'operator': 'equal',
                                                      'value': 'value.login'},
                                                     {'id': 'method_clean',
                                                      'field': 'method_clean',
                                                      'type': 'string',
                                                      'operator': 'begins_with',
                                                      'value': 'value.login'},
                                                     {'id': 'method_clean',
                                                      'field': 'method_clean',
                                                      'type': 'string',
                                                      'operator': 'ends_with',
                                                      'value': 'value.login'},
                                                     {'id': 'method_clean',
                                                      'field': 'method_clean',
                                                      'type': 'string',
                                                      'operator': 'contains',
                                                      'value': 'value.login'},
                                                     {'id': 'ip_address',
                                                      'field': 'ip_address',
                                                      'type': 'string',
                                                      'operator': 'is_null',
                                                      'value': None},
                                                     {'id': 'ip_isp',
                                                      'field': 'ip_isp',
                                                      'type': 'string',
                                                      'operator': 'is_empty',
                                                      'value': None}
                                                     ]}]},
                      'Rule3': {'condition': 'AND',
                                'rules': [{'id': 'method_clean',
                                           'field': 'method_clean',
                                           'type': 'string',
                                           'operator': 'not_begins_with',
                                           'value': 'value.login'},
                                          {'id': 'method_clean',
                                           'field': 'method_clean',
                                           'type': 'string',
                                           'operator': 'not_ends_with',
                                           'value': 'value.login'},
                                          {'id': 'method_clean',
                                           'field': 'method_clean',
                                           'type': 'string',
                                           'operator': 'not_contains',
                                           'value': 'value.login'},
                                          {'condition': 'OR',
                                           'rules': [{'id': 'ip_address',
                                                      'field': 'ip_address',
                                                      'type': 'string',
                                                      'operator': 'is_not_null',
                                                      'value': None},
                                                     {'id': 'ip_isp',
                                                      'field': 'ip_isp',
                                                      'type': 'string',
                                                      'operator': 'is_not_empty',
                                                      'value': None},
                                                     ],
                                           }]},
                      'Rule4': {'condition': 'AND',
                                'rules': [{'id': 'forwarder_address',
                                           'field': 'forwarder_address',
                                           'type': 'boolean',
                                           'input': 'radio',
                                           'operator': 'equal',
                                           'value': '0'},
                                          {'id': 'is_shipping_billing_address_same',
                                           'field': 'is_shipping_billing_address_same',
                                           'type': 'boolean',
                                           'input': 'radio',
                                           'operator': 'equal',
                                           'value': '1'}]},
                      'Rule5': {'condition': 'AND',
                                'rules': [{'id': 'ad_price_type',
                                           'field': 'ad_price_type',
                                           'type': 'string',
                                           'operator': 'in',
                                           'value': 'value.FREE,NEGOTIATION'},
                                          {'id': 'ad_price_type',
                                           'field': 'ad_price_type',
                                           'type': 'string',
                                           'operator': 'not_in',
                                           'value': 'value.FOO,BAR'}
                                          ]},
                      'Rule6': {'condition': 'AND',
                                'rules': [{'id': 'ip_country_iso_code',
                                           'field': 'ip_country_iso_code',
                                           'type': 'string',
                                           'operator': 'not_equal',
                                           'value': 'field.billing_country',
                                           },
                                          {'id': 'country_id',
                                           'field': 'country_id',
                                           'type': 'string',
                                           'operator': 'equal',
                                           'value': 'field.ip_country_iso_code',
                                           }]}}

In [3]:
# old_scores = {
#         'Rule1': -100,
#         'Rule2': -100,
#         'Rule3': -100,
#         'Rule4': -100,
#         'Rule5': -100,
#         'Rule6': -100,
#     }

In [12]:
new_scores = {
        'Rule1': -50,
        'Rule2': -50,
        'Rule3': -50,
        'Rule4': -50,
        'Rule5': -50,
        'Rule6': -50,
    }

In [13]:
from system_config_generation.update_existing_configs import UpdateExistingConfigs

In [14]:
u = UpdateExistingConfigs(rule_configs=rule_configs, updated_conditions=new_conditions, updated_scores=new_scores)

In [15]:
u.update()

{'Rule1': {'createdBy': 'argo@simility.com',
  'createdOn': '2020-12-16T15:00:17',
  'appPrefix': 'james_testing',
  'entityName': 'transaction',
  'name': 'Rule1',
  'conditions': '{"condition": "AND", "rules": [{"condition": "OR", "rules": [{"id": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_1day", "type": "double", "operator": "greater_or_equal", "value": "value.120.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_7day", "type": "double", "operator": "greater", "value": "value.240.0"}, {"id": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "field": "payer_id.sum_approved_txn_amt_per_paypalid_30day", "type": "double", "operator": "less_or_equal", "value": "value.1000.0"}]}, {"id": "num_items", "field": "num_items", "type": "integer", "operator": "equal", "value": "value.2"}]}',
  'score': -50,
  'status': 'ACTIVE',
  'isAutoGenerated': True,
  'modifiedBy'

In [8]:
rule_configs_dict = dict((k, json.loads(v)) for k, v in c.rule_configs.items())

NameError: name 'json' is not defined

In [9]:
rule_configs_dict

NameError: name 'rule_configs_dict' is not defined

In [153]:
update = UpdateExistingConfigs(rule_configs=rule_configs_dict, updated_conditions=updated_system_dicts, updated_scores=new_scores)

In [154]:
update.update()

{'Rule1': '{"createdBy": "argo@simility.com", "createdOn": "2020-12-16T15:00:17", "appPrefix": "james_testing", "entityName": "transaction", "name": "Rule1", "conditions": "{\\"condition\\": \\"AND\\", \\"rules\\": [{\\"condition\\": \\"OR\\", \\"rules\\": [{\\"id\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_1day\\", \\"field\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_1day\\", \\"type\\": \\"double\\", \\"operator\\": \\"greater_or_equal\\", \\"value\\": \\"value.120.0\\"}, {\\"id\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_7day\\", \\"field\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_7day\\", \\"type\\": \\"double\\", \\"operator\\": \\"greater\\", \\"value\\": \\"value.240.0\\"}, {\\"id\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_30day\\", \\"field\\": \\"payer_id.sum_approved_txn_amt_per_paypalid_30day\\", \\"type\\": \\"double\\", \\"operator\\": \\"less_or_equal\\", \\"value\\": \\"value.1000.0\\"}]}, {\\"id\\": \\"num_items\\", \\"field\\": \\"num_it

In [158]:
updated_rule_configs_dict = dict((k, json.loads(v)) for k, v in update.updated_rule_configs.items())

In [164]:
for rule_name, rule_config in updated_rule_configs_dict.items():
    assert json.loads(rule_config['conditions']) == updated_system_dicts[rule_name]
    assert rule_config['score'] == -50